## Faiss(파이스)

- 대용량 데이터 (수백만개 벡터)
- 최고 성능이 필요한 경우
- 메모리 기반 빠른 검색
- 임시 작업용

In [ ]:
# !pip install faiss-cpu

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI
from langchain.callbacks import get_openai_callback
from IPython.display import JSON

from dotenv import load_dotenv
import os

load_dotenv()
api_key= os.getenv('OPENAI_API_KEY')


/opt/anaconda3/envs/llm_env/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
text = """생성형 인공지능 또는 생성형 AI는 프롬프트에 대응하여 텍스트, 이미지, 기타 미디어를 생성할 수 있는 일종의 인공지능 시스템이다.
단순히 기존 데이터를 분석하는 것이 아닌, 새로운 콘텐츠를 만드는 데 초점을 맞춘 인공지능 분야를 말한다. 2022년경부터 본격적으로 유명해지기 시작했다.
데이터 원본을 통한 학습으로 소설, 이미지, 비디오, 코딩, 음악, 미술 등 다양한 콘텐츠 생성에 이용된다. 한국에서는 2022년 Novel AI 등, 그림 인공지능의 등장으로 주목도가 높아졌으며, 해외에서는 미드저니, 챗GPT등 여러 모델을 잇달아 공개하면서 화제의 중심이 되었다.
보통 딥러닝 인공지능은 학습 혹은 결과 출력 전 원본 자료를 배열 자료형[2] 숫자 데이터로 변환하는 인코딩 과정이 중요한데, 생성 AI의 경우 인공지능의 출력 데이터를 역으로 그림, 글 등의 원하는 형태로 변환시켜주는 디코딩 과정 또한 필요하다.
사실상 인공지능의 대중화를 이끈 기술로써, 해당 기술이 인공지능에 대한 사람들의 전반적인 인식을 매우 크게 바꿔놨다고 해도 과언이 아니다.
"""

texts = [
    "생성형 AI는 기존 데이터를 학습하여 새로운 콘텐츠를 자동으로 생성하는 인공지능 기술이다.",
    "생성형 인공지능이란, 사용자의 명령에 따라 텍스트, 이미지, 음성 등을 창조해내는 모델이다.",
    "전통적인 AI는 규칙 기반으로 작동하지만, 생성형 AI는 학습한 패턴을 기반으로 창의적인 출력을 생성한다.",
    "생성형 AI는 마케팅 콘텐츠 제작, 음악 생성, 코딩 보조 등 다양한 분야에 활용되고 있다.",
    "ChatGPT는 생성형 AI 모델 중 하나로, 대화형 응답을 생성하는 데 최적화되어 있다.",
    "이미지 생성 AI인 DALL·E는 텍스트 설명을 바탕으로 고해상도 이미지를 생성한다.",
    "2023년 이후 생성형 AI 시장은 급성장하고 있으며, 글로벌 기업들이 기술 확보에 나서고 있다.",
    "생성형 AI는 창작의 패러다임을 바꾸며, 윤리적 문제나 저작권 이슈도 함께 제기되고 있다.",
    "생성형 AI는 트랜스포머 기반의 대규모 언어 모델(LLM)을 핵심 기술로 삼고 있다.",
    "분류 모델, 추천 시스템 등은 생성형 AI가 아닌 전통적인 분석형 AI로 분류된다.",
]

# 리스트를 하나의 문자열로 결합
full_text = "\n".join(texts)

# CharacterTextSplitter을 이용한 청킹
splitter = CharacterTextSplitter(
    separator="\n", chunk_size=300, chunk_overlap=50, length_function=len
)

# 주어진 문장 청킹
chunks = splitter.split_text(full_text)
# chunks = splitter.split_text(text)
print(chunks)

['생성형 AI는 기존 데이터를 학습하여 새로운 콘텐츠를 자동으로 생성하는 인공지능 기술이다.\n생성형 인공지능이란, 사용자의 명령에 따라 텍스트, 이미지, 음성 등을 창조해내는 모델이다.\n전통적인 AI는 규칙 기반으로 작동하지만, 생성형 AI는 학습한 패턴을 기반으로 창의적인 출력을 생성한다.\n생성형 AI는 마케팅 콘텐츠 제작, 음악 생성, 코딩 보조 등 다양한 분야에 활용되고 있다.\nChatGPT는 생성형 AI 모델 중 하나로, 대화형 응답을 생성하는 데 최적화되어 있다.', 'ChatGPT는 생성형 AI 모델 중 하나로, 대화형 응답을 생성하는 데 최적화되어 있다.\n이미지 생성 AI인 DALL·E는 텍스트 설명을 바탕으로 고해상도 이미지를 생성한다.\n2023년 이후 생성형 AI 시장은 급성장하고 있으며, 글로벌 기업들이 기술 확보에 나서고 있다.\n생성형 AI는 창작의 패러다임을 바꾸며, 윤리적 문제나 저작권 이슈도 함께 제기되고 있다.\n생성형 AI는 트랜스포머 기반의 대규모 언어 모델(LLM)을 핵심 기술로 삼고 있다.\n분류 모델, 추천 시스템 등은 생성형 AI가 아닌 전통적인 분석형 AI로 분류된다.']


In [4]:
# 임베딩 모델
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)

# 단순 텍스트 배열 (strings) 임베딩을 FAISS FAISS(Vector Store)에 저장
faiss_store = FAISS.from_texts(chunks, embeddings)  # 문자열 리스트

/var/folders/v9/qpg5js816p79wmppzgfqcz9w0000gn/T/ipykernel_37576/2615808152.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)


In [5]:
print(faiss_store)

# 기본이 메모리 저장됨, 수동 저장 필요
faiss_store.save_local("./faiss_backup")

In [ ]:
question = "생성형 AI란?"
references = faiss_store.similarity_search(question)

- chain_type
    - stuff: 문서가 짧거나 3~4개 이하일 때 적합
    - map_reduce, refine: 문서가 많거나 길 때
    - map_rerank: **정확성(precision)**이 중요할 때

    ```Markdown
    | chain_type    | 설명                                                           |
    | -------------- | ------------------------------------------------------------- |
    | `"stuff"`      | 모든 문서를 한 번에 붙여서 질문과 함께 LLM에 전달 (**가장 단순하고 빠름**)          |
    | `"map_reduce"` | 각 문서에 대해 개별적으로 응답 생성 → 요약 (Map 단계), 전체 요약 생성 (Reduce 단계) |
    | `"refine"`     | 첫 문서로 초기 답 생성 → 다음 문서를 순차적으로 읽으며 답을 점점 보완                |
    | `"map_rerank"` | 문서마다 답과 점수를 생성 → 가장 높은 점수의 응답 선택                         |
    ```

In [1]:
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)

chain = load_qa_chain(llm, chain_type="stuff") #Q&A Interface, 하지만 모든 document를 가져온다
response = chain.run(input_documents=references, question=question)
print(response)

NameError: name 'ChatOpenAI' is not defined

In [ ]:
with get_openai_callback() as cb:
    
    print(cb)

생성형 인공지능 또는 생성형 AI는 프롬프트에 대응하여 텍스트, 이미지, 기타 미디어를 생성할 수 있는 인공지능 시스템입니다. 이는 기존 데이터를 분석하는 것이 아니라 새로운 콘텐츠를 만드는 데 초점을 맞춘 인공지능 분야를 의미합니다. 해당 기술은 2022년경부터 본격적으로 유명해지기 시작했습니다.
